Install Deps

In [17]:
import numpy as np
import tensorflow as tf
import tflearn
import spacy
nlp = spacy.load('en')
import re
from nltk.util import ngrams, trigrams
import csv
from tflearn.data_utils import to_categorical

Load the model

In [3]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    net = tflearn.input_data([None, 1200])                          # Input
    net = tflearn.fully_connected(net, 200, activation='ReLU')      # Hidden
    net = tflearn.fully_connected(net, 25, activation='ReLU')      # Hidden
    net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
    model = tflearn.DNN(net)

    return model

model = build_model()


In [4]:

model.load('./model.tfl')

Import the vocab index

In [5]:
import csv
word2idx = {}
for key, val in csv.reader(open("./vocabindex.csv")):
    word2idx[key] = int(val)
    
word2idx

{'ADP>DET>NOUN': 0,
 'NOUN>ADP>DET': 1,
 'ADJ>NOUN>PUNCT': 2,
 'DET>ADJ>NOUN': 3,
 'DET>NOUN>ADP': 4,
 'ADJ>NOUN>ADP': 5,
 'NOUN>ADP>NOUN': 6,
 'DET>NOUN>PUNCT': 7,
 'ADP>ADJ>NOUN': 8,
 'ADP>DET>ADJ': 9,
 'VERB>DET>NOUN': 10,
 'NOUN>NOUN>PUNCT': 11,
 'VERB>ADP>DET': 12,
 'DET>NOUN>VERB': 13,
 'ADP>NOUN>PUNCT': 14,
 'VERB>VERB>ADP': 15,
 'NOUN>ADP>ADJ': 16,
 'NOUN>VERB>VERB': 17,
 'ADJ>NOUN>VERB': 18,
 'ADJ>ADJ>NOUN': 19,
 'ADJ>NOUN>NOUN': 20,
 'NOUN>ADP>PROPN': 21,
 'DET>NOUN>NOUN': 22,
 'VERB>ADJ>NOUN': 23,
 'PROPN>PROPN>PUNCT': 24,
 'NOUN>NOUN>ADP': 25,
 'VERB>DET>ADJ': 26,
 'NOUN>VERB>ADP': 27,
 'NOUN>PUNCT>NOUN': 28,
 'NOUN>PUNCT>VERB': 29,
 'VERB>ADP>NOUN': 30,
 'ADP>NOUN>ADP': 31,
 'ADP>NUM>PUNCT': 32,
 'NOUN>ADP>NUM': 33,
 'ADP>DET>PROPN': 34,
 'NOUN>PUNCT>CONJ': 35,
 'NOUN>CONJ>NOUN': 36,
 'ADP>PROPN>PUNCT': 37,
 'VERB>NOUN>ADP': 38,
 'PROPN>PROPN>PROPN': 39,
 'DET>PROPN>PROPN': 40,
 'VERB>ADP>PROPN': 41,
 'NOUN>NOUN>VERB': 42,
 'NUM>NOUN>PUNCT': 43,
 'VERB>PART>VERB': 44,
 'VE

Import the dataprep methods

In [6]:
def textStringToPOSArray(text):
    doc = nlp(text)
    tags = []
    for word in doc:
        tags.append(word.pos_)
    return tags

def find_ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

def getPOSTrigramsForTextString(text):
    tags = textStringToPOSArray(text)
    tgrams = list(trigrams(tags))
    return tgrams

def trigramsToDictKeys(trigrams):
    keys = []
    for trigram in trigrams:
        keys.append('>'.join(trigram))
    return keys

def textToTrigrams(text): 
    return trigramsToDictKeys(getPOSTrigramsForTextString(text))

def text_to_vector(text):
    wordVector = np.zeros(1200)
    for word in textToTrigrams(text):
        index = word2idx.get(word, None)
        if index != None:
            wordVector[index] += 1
    return wordVector

def test_sentence(sentence):
    positive_prob = model.predict([text_to_vector(sentence)])[0][1]
    print('Sentence: {}'.format(sentence))
    print('P(positive) = {:.3f} :'.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')
    return positive_prob

Try it out!

In [40]:
test_sentence("Since she never saw that movie.") 

Sentence: Since she never saw that movie.
P(positive) = 1.000 : Positive


0.9999998807907104

In [39]:
test_sentence("We should invite her, since she never saw that movie.")

Sentence: We should invite her, since she never saw that movie.
P(positive) = 1.000 : Positive


1.0

In [38]:
test_sentence("Standing on the edge of the cliff looking down.")

Sentence: Standing on the edge of the cliff looking down.
P(positive) = 0.998 : Positive


0.998099148273468

In [41]:
test_sentence("Donald")

Sentence: Donald
P(positive) = 0.993 : Positive


0.9931533336639404

In [11]:
test_sentence("Standing on the edge of the cliff looking down, Donald could see a boat.")

Sentence: Standing on the edge of the cliff looking down, Donald could see a boat.
P(positive) = 0.918 : Positive


0.918357789516449

In [12]:
data = ["Ate the Cake.",
"Washed the dishes.",
"Did my homework.",
"Ran a mile.",
"Went fishing.",
"Made the bed today.",
"Traveled to Europe.",
"Saw a movie.",
"Sharpened the pencil.",
"Playing the guitar.",
"Completing the assignment.",
"Celebrating the birthday.",
"Always working before unfriendly crowds.",
"Dragging his heels a little in the fine dust.",
"Creeping warily under the shadows of tottering walls.",
"Watching the mower at work, feeling the warm, soft sunshine seep into his bones.",
"Worrying that a meteor or chunk of space debris will conk her on the head.",
"Invented by an Indiana housewife in.",
"Driven by a steam engine.",
"Stuck in the back of the closet behind the obsolete computer.",
"Punctuated by the white lightning of hunger and the flickering of fear.",
"Caged in a dingy mesh of fire escapes.",
"Propelled by short sweeps of its crescent tail.",
"And dashed through the downpour as raindrops softened the hairspray shell holding her elaborate coif in place.",
"But knew that all of his effort would prove useless in the long run.",
"Took the thick book and, with a heavy sigh, loaded it on top of her research pile.",
"And let the raccoons, opossums, and armadillos that visit the yard eat the leftovers..",
"A red convertible with fancy rims and fuzzy dice hanging from the rearview mirror.",
"The unprepared student who was always begging for an extra pencil and a couple sheets of blank paper.",
"A slacker wasting his afternoon in front of the television.",
"A dog around whom people need to guard their fingers and food.",
"A beautiful day, perfect for a picnic.",
"The best teacher in the world.",
"A world class chess player.",
"A young Peace Corps worker.",
"The student slurping hot soup.",
"A young woman whose hair reaches her waist.",
"To explain why he had brought Squeeze, his seven-foot pet python, to Mr Parker’s English class.",
"To figure out what we’re doing wrong.",
"To keep the floor clean.",
"To stay in shape.",
"To gain her mother’s approval.",
"To make more friends.",
"To get to know people in the neighborhood.",
"To smash a spider.",
"To kick the ball past the dazed goalie.",
"To lick the grease from his shiny fingers despite the disapproving glances of his girlfriend Gloria.",
"To finish her shift without spilling another pizza into a customer’s lap.",
"To keep their customers happy.",
"To figure out how to improve my painting.",
"How to do it.",
"Including the dog with three legs and the cat with one eye.",
"Such as leaving the stove on and teasing mean dogs.",
"After the movie is over.",
"After the rain stops.",
"Although I’m not hungry.",
"As I was walking home.",
"As if it were by divine providence.",
"As long as you behave yourself.",
"As soon as I finish my homework.",
"As though she really knew how to sew.",
"As his ex-girlfriend Gigi chased him down the interstate.",
"Because he was so angry.",
"Because I like it.",
"Because his car is in the shop.",
"Before I go to bed.",
"Even little kids.",
"If you do that again.",
"If you want to go.",
"Even though I don’t like pasta.",
"In case I’m late.",
"In that both cats and dogs are pets.",
"In order that we don’t lose track of attendance.",
"Insofar as we are friends.",
"Just as I got home.",
"No matter how hard I try.",
"Now that you’re here.",
"Once I finish my homework.",
"Provided that you behave.",
"Rather than bake a cake.",
"So I don’t forget.",
"So that we don’t get lost.",
"Than.",
"That thing you do.",
"Though she doesn’t speak French.",
"Till I get home from work.",
"Unless I earn enough money.",
"Until I’m old enough.",
"When you finally take the test.",
"Whenever I ask how you’re doing.",
"Where you go to school.",
"Whereas chimpanzees use tools.",
"Wherever we end up.",
"Whether or not we succeed.",
"Since you asked.",
"Since we finished the test.",
"Since Sarah ate cake.",
"While we were fishing.",
"While some primates live in groups.",
"While I was sleeping.",
"Which makes me hungry.",
"Whichever you choose.",
"Who participated in the program.",
"Whoever wants to improve the writing skill.",
"Whom I met yesterday.",
"Whomever you care about.",
"Whose water bottle is on the table.",
"Where the wind blows.",
"When I finish my homework.",
"Why this happened."]

In [13]:
len(data)

110

In [31]:
count = 0
for sentence in data:
    if test_sentence(sentence) < 0.5:
        count += 1

Sentence: Ate the Cake.
P(positive) = 0.000 : Negative
Sentence: Washed the dishes.
P(positive) = 0.000 : Negative
Sentence: Did my homework.
P(positive) = 0.000 : Negative
Sentence: Ran a mile.
P(positive) = 0.000 : Negative
Sentence: Went fishing.
P(positive) = 0.000 : Negative
Sentence: Made the bed today.
P(positive) = 0.000 : Negative
Sentence: Traveled to Europe.
P(positive) = 0.000 : Negative
Sentence: Saw a movie.
P(positive) = 0.000 : Negative
Sentence: Sharpened the pencil.
P(positive) = 0.000 : Negative
Sentence: Playing the guitar.
P(positive) = 0.000 : Negative
Sentence: Completing the assignment.
P(positive) = 0.000 : Negative
Sentence: Celebrating the birthday.
P(positive) = 0.000 : Negative
Sentence: Always working before unfriendly crowds.
P(positive) = 0.000 : Negative
Sentence: Dragging his heels a little in the fine dust.
P(positive) = 0.000 : Negative
Sentence: Creeping warily under the shadows of tottering walls.
P(positive) = 0.000 : Negative
Sentence: Watching t

In [32]:
count

110

In [30]:
for sentence in data:
  print(text_to_vector(sentence))
  print(0.)
  model.fit([text_to_vector(sentence)], to_categorical([0], 2))

[ 0.  0.  0. ...,  0.  0.  0.]
0.0
---------------------------------
Run id: 9427QD
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 1
Validation samples: 0
--
Training Step: 37365  | total loss: 0.02047 | time: 0.004s
| SGD | epoch: 015 | loss: 0.02047 -- iter: 1/1
--
Training Step: 37366  | total loss: 0.01848 | time: 0.003s
| SGD | epoch: 016 | loss: 0.01848 -- iter: 1/1
--
Training Step: 37367  | total loss: 0.01669 | time: 0.003s
| SGD | epoch: 017 | loss: 0.01669 -- iter: 1/1
--
Training Step: 37368  | total loss: 0.01507 | time: 0.004s
| SGD | epoch: 018 | loss: 0.01507 -- iter: 1/1
--
Training Step: 37369  | total loss: 0.01362 | time: 0.003s
| SGD | epoch: 019 | loss: 0.01362 -- iter: 1/1
--
Training Step: 37370  | total loss: 0.01231 | time: 0.004s
| SGD | epoch: 020 | loss: 0.01231 -- iter: 1/1
--
Training Step: 37371  | total loss: 0.01112 | time: 0.004s
| SGD | epoch: 021 | loss: 0.01112 -- iter: 1/1
--
Training Step: 37372  | total los

In [29]:
model.fit([text_to_vector(data[0])], to_categorical([0], 2))

---------------------------------
Run id: 9GW1UN
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 1
Validation samples: 0
--
Training Step: 37355  | total loss: 0.05602 | time: 0.035s
| SGD | epoch: 005 | loss: 0.05602 -- iter: 1/1
--
Training Step: 37356  | total loss: 0.05156 | time: 0.003s
| SGD | epoch: 006 | loss: 0.05156 -- iter: 1/1
--
Training Step: 37357  | total loss: 0.04652 | time: 0.003s
| SGD | epoch: 007 | loss: 0.04652 -- iter: 1/1
--
Training Step: 37358  | total loss: 0.04197 | time: 0.002s
| SGD | epoch: 008 | loss: 0.04197 -- iter: 1/1
--
Training Step: 37359  | total loss: 0.03787 | time: 0.003s
| SGD | epoch: 009 | loss: 0.03787 -- iter: 1/1
--
Training Step: 37360  | total loss: 0.03417 | time: 0.003s
| SGD | epoch: 010 | loss: 0.03417 -- iter: 1/1
--
Training Step: 37361  | total loss: 0.03083 | time: 0.003s
| SGD | epoch: 011 | loss: 0.03083 -- iter: 1/1
--
Training Step: 37362  | total loss: 0.02783 | time: 0.003s
| SGD | e

In [23]:
text_to_vector(data[0])

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [35]:
to_categorical([1], 2)

array([[ 0.,  1.]])

In [28]:
to_categorical([0, 1, 3], 2)

IndexError: index 3 is out of bounds for axis 1 with size 2

In [33]:
jjdata = ["John ate the cake. ",
"Donald and Ryan washed the dishes.",
"Did my homework.",
"I ran a mile.",
"We went fishing.",
"Sharon made the bed today.",
"They traveled to Europe.",
"You saw a movie.",
"The student sharpened the pencil.",
"I’m playing the guitar.",
"She is completing the assignment.",
"We are celebrating the birthday.",
"The artist is always working before unfriendly crowds.",
"Brent’s dragging his heels a little in the fine dust.",
"They were creeping warily under the shadows of tottering walls.",
"He’s watching the mower at work, feeling the warm, soft sunshine seep into his bones.",
"Jenny is worrying that a meteor or chunk of space debris will conk her on the head.",
"It was invented by an Indiana housewife in the winter.",
"The train is driven by a steam engine.",
"The book is stuck in the back of the closet behind the obsolete computer.",
"Every day is punctuated by the white lightning of hunger and the flickering of fear.",
"You’re caged in a dingy mesh of fire escapes.",
"The bird’s flight is propelled by short sweeps of its crescent tail.",
"Tina dashed through the downpour as raindrops softened the hairspray shell holding her elaborate coif in place.",
"Ryan tried, but knew that all of his efforts would prove useless in the long run.",
"The professor took the thick book and, with a heavy sigh, loaded it on top of her research pile.",
"Just relax and let the raccoons, opossums, and armadillos that visit the yard eat the leftovers.",
"I drive a red convertible with fancy rims and fuzzy dice hanging from the rearview mirror.",
"The unprepared student who was always begging for an extra pencil and a couple sheets of blank paper was failing the class.",
"I feel like a slacker wasting his afternoon in front of the television.",
"Maxwell’s a dog around whom people need to guard their fingers and food.",
"This is a beautiful day, perfect for a picnic.",
"The best teacher in the world is in this room.",
"He’s a world class chess player.",
"My best friend is a young Peace Corps worker.",
"Who’s the student slurping hot soup?",
"Rachel is a young woman whose hair reaches her waist.",
"Tim had to explain why he had brought Squeeze, his seven-foot pet python, to Mr Parker's English class.",
"It’s time to figure out what we’re doing wrong.",
"To keep the floor clean, use soap.",
"I want to stay in shape.",
"She wants to gain her mother’s approval.",
"Bryan needs to make more friends.",
"To get to know people in the neighborhood, do activities.",
"I smash a spider.",
"I’ll try to kick the ball past the dazed goalie.",
"Bryan decided to lick the grease from his shiny fingers despite the disapproving glances of his girlfriend Gloria.",
"Tammy finished her shift without spilling another pizza into a customer's lap.",
"Businesses try to keep their customers happy.",
"I want to figure out how to improve my painting.",
"I know how to do it.",
"Including the dog with three legs and the cat with one eye.",
"I make mistakes, such as leaving the stove on and teasing mean dogs.",
"After the movie is over, they leave.",
"We’ll play after the rain stops.",
"Although I’m not hungry, I eat.",
"I laughed as I was walking home.",
"As if it were by divine providence, it happened.",
"You’ll get candy as long as you behave yourself.",
"Let’s go as soon as I finish my homework.",
"She offered, as though she really knew how to sew.",
"He cried as his ex-girlfriend Gigi chased him down the interstate.",
"Because he was so angry, Steven yelled.",
"I bought it because I like it.",
"Because his car is in the shop, John had to walk.",
"Before I go to bed, I pray.",
"Even little kids can do addition.",
"If you do that again, you’re in trouble.",
"We can go if you want to go.",
"Even though I don’t like pasta, I’ll eat it.",
"Leave without me, in case I’m late.",
"In that both cats and dogs are pets, they’re similar.",
"Let’s take notes, in order that we don’t lose track of attendance.",
"I like you insofar as we are friends.",
"You arrived just as I got home.",
"No matter how hard I try, I fail.",
"Now that you’re here, Bill’s happy.",
"I’ll do it once I finish my homework.",
"You’ll get candy provided that you behave.",
"Rather than bake a cake, I want to dance.",
"I write so I don’t forget.",
"Let’s bring a map s that we don’t get lost.",
"I’m better than that.",
"That thing you do is funny.",
"Though she doesn’t speak French, she acts French.",
"I’ll talk till I get home from work.",
"I won’t quit unless I earn enough money.",
"You can wait until I’m old enough.",
"We’ll see when you finally take the test.",
"Whenever I ask how you’re doing, you say “fine.”",
"I forget where you go to school.",
"Ants only use their bodies, whereas chimpanzees use tools.",
"Wherever we end up, we’ll be together.",
"It doesn’t matter whether or not we succeed.",
"I’ll tell you, since you asked.",
"Since we finished the test, let’s celebrate.",
"Bryan is hungry, since Sarah ate cake.",
"While we were fishing, a fish escaped.",
"Humans live alone, while some primates live in groups.",
"While I was sleeping, I dreamt of you.",
"I skipped breakfast, which makes me hungry.",
"Whichever you choose, choose well.",
"I know a guy who participated in the program.",
"Whoever wants to improve the writing skill uses Quill.",
"That’s the girl whom I met yesterday.",
"Be nice to whomever you care about.",
"Ryan, whose water bottle is on the table, is late.",
"Brad lives where the wind blows.",
"When I finish my homework, I go to sleep.",
"Who knows why this happened."]


In [34]:
len(jjdata)

110

In [36]:
for sentence in jjdata:
  print(text_to_vector(sentence))
  print(0.)
  model.fit([text_to_vector(sentence)], to_categorical([1], 2))

[ 0.  0.  0. ...,  0.  0.  0.]
0.0
---------------------------------
Run id: PKE0SS
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 1
Validation samples: 0
--
Training Step: 38465  | total loss: 0.00000 | time: 0.003s
| SGD | epoch: 1115 | loss: 0.00000 -- iter: 1/1
--
Training Step: 38466  | total loss: 1.15629 | time: 0.003s
| SGD | epoch: 1116 | loss: 1.15629 -- iter: 1/1
--
Training Step: 38467  | total loss: 1.16820 | time: 0.003s
| SGD | epoch: 1117 | loss: 1.16820 -- iter: 1/1
--
Training Step: 38468  | total loss: 1.05139 | time: 0.004s
| SGD | epoch: 1118 | loss: 1.05139 -- iter: 1/1
--
Training Step: 38469  | total loss: 0.94626 | time: 0.003s
| SGD | epoch: 1119 | loss: 0.94626 -- iter: 1/1
--
Training Step: 38470  | total loss: 0.85164 | time: 0.003s
| SGD | epoch: 1120 | loss: 0.85164 -- iter: 1/1
--
Training Step: 38471  | total loss: 0.76649 | time: 0.003s
| SGD | epoch: 1121 | loss: 0.76649 -- iter: 1/1
--
Training Step: 38472  | to

In [37]:
model.save("./model2.tfl")

INFO:tensorflow:/Users/donald/Programming/Python/Quill-NLP-Tools-and-Datasets/model2.tfl is not in all_model_checkpoint_paths. Manually adding it.
